In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

/Users/anuragsingh/Documents/GitHub/Mr-analyzer/analyzerenv/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [3]:
extracted_data = load_pdf("pdfs/")

In [4]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [5]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 4


In [6]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [7]:
embeddings = download_hugging_face_embeddings()

/Users/anuragsingh/Documents/GitHub/Mr-analyzer/analyzerenv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [8]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [9]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [11]:
#Initializing the Pinecone
import os
from pinecone import Pinecone
pinecone_api=os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api)
index = pc.Index("mr-analyzer")
index_name="mr-analyzer"
index.delete(delete_all=True)

from langchain.vectorstores import Pinecone as PC
docs_chunks = [t.page_content for t in text_chunks]
pinecone_index = PC.from_texts(
    docs_chunks,
    embeddings,
    index_name='mr-analyzer'
)

In [12]:
#If we already have an index we can load it like this
docsearch=PC.from_existing_index(index_name, embeddings)

query = "What anurag know in tech"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='collaboration) \nd.\nREADME\nfile\nmaintenance \n2.\nCode\nquality \na.\nModularity\nfollowed. \nb.\nBasic\ncode\npractices\nlike\ncommenting,\nproper\nvariable\nand\nfunction\nnaming,\netc. \nare\nfollowed.'), Document(page_content='To\nensure\na\nfair\nand\ncomprehensive\nevaluation\nof\nall\nprojects,\nthe\nfollowing\njudging \ncriteria\nwill\nbe\nused:\n1.\nInnovation\nand\nCreativity \na.\nOriginality:\nHow\nunique\nand\noriginal\nis\nthe\nidea? \nb.\nCreativity:\nDoes\nthe\nproject\noffer\na\nnovel\napproach\nto\nsolving\nthe\nproblem? \n2.\nTechnical\nImplementation \na.\nComplexity:\nHow\nchallenging\nwas\nthe\ntechnical\nimplementation? \nb.\nFunctionality:\nDoes\nthe\nproject\nwork\nas\nintended?\nAre\nthere\nany\nbugs\nor\nissues? \nc.\nScalability:\nCan\nthe\nproject\nbe\nscaled\nfor'), Document(page_content='be\nscaled\nfor\nlarger\naudiences\nor\ndifferent\nuse\ncases? \n3.\nUser\nExperience\nand\nDesign \na.\nUsability:\nIs\nthe\nproject\ne

In [13]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [14]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI
google_gemini_api=os.getenv("GOOGLE_API_KEY")
llm_model=ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=google_gemini_api)

I0000 00:00:1722507553.296788  142236 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache
I0000 00:00:1722507553.308472  142236 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722507553.311562  142236 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [16]:
qa=RetrievalQA.from_chain_type(
    llm=llm_model, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [17]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

/Users/anuragsingh/Documents/GitHub/Mr-analyzer/analyzerenv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Response :  1. Innovation and Creativity
2. Technical Implementation
3. User Experience and Design
4. Impact and Usefulness
5. Presentation and Communication
Response :  How is the project idea communicated?
